# Task 1

Линейная модель:

$Y = X\beta + \varepsilon$, где 
- $\vec\varepsilon \sim \mathcal{N}(0, \text{diag}(\sigma^2))$
- $X\in M_{n \times 3}$
- $Y\in M_{n\times 1}$

Столбцы $X$ - признаки: продолжительность, танцевальность и энергичность;

$Y$ - столбец значений популярности.

Подготовим данные.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('songs.csv')

X = df[['song_duration_ms', 'danceability', 'energy']].values
y = df['song_popularity'].values

print("X:", X.shape)
print("y:", y.shape)

X: (18835, 3)
y: (18835,)


In [ ]:
def solve(X, y):
    n_samples = X.shape[0]
    
    # 1) simple
    XtX = X.T @ X
    Xty = X.T @ y
    beta = np.linalg.solve(XtX, Xty)
    
    # 2) SVD
    # U, s, Vt = np.linalg.svd(X, full_matrices=False)
    # beta = Vt.T @ np.diag(1.0 / s) @ U.T @ y
    
    y_pred = X @ beta    
    rss = np.sum((y - y_pred)**2)
    
    n_features = X.shape[1]
    df = n_samples - n_features
    
    # несмещ.
    residual_variance = rss / df
    
    return beta, residual_variance

print(*solve(X, y))

[6.91437859e-05 4.53102909e+01 1.25473532e+01] 524.3467426503969
